In [1]:
import sys
import re
import numpy as np
import ast

In [2]:
sys.path.append('MLSTM-FCN/')

In [3]:
resultsFilename='results_data_FINAL.txt'

In [4]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

DATASET_INDEX_0 = 52
DATASET_INDEX_1 = 53
DATASET_INDEX_2 = 54
DATASET_INDEX_3 = 55

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX_0]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX_0]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX_0]

TRAINABLE = True

Using TensorFlow backend.
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/t

In [6]:
def generate_model():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    # stride = 10

    # x = Permute((2, 1))(ip)
    # x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
    #            kernel_initializer='he_uniform')(x)  # (None, variables / stride, timesteps)
    # x = Permute((2, 1))(x)

    #ip1 = K.reshape(ip,shape=(MAX_TIMESTEPS,MAX_NB_VARIABLES))
    #x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = AttentionLSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='softmax')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [9]:
def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [10]:
n_epochs=250

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 4096)      0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 4096, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 4096, 128)    1152        permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4096, 128)    512         conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation

In [ ]:
if __name__ == "__main__":
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 0", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_0, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_0, dataset_prefix='eeg2_attention', batch_size=128)
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 1", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_1, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_1, dataset_prefix='eeg2_attention', batch_size=128)  
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 2", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_2, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_2, dataset_prefix='eeg2_attention', batch_size=128)
    with open(resultsFilename, "a") as text_file:
        print(f"Current Fold: 3", file=text_file)
    model = generate_model_2()
    train_model(model, DATASET_INDEX_3, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
    evaluate_model(model, DATASET_INDEX_3, dataset_prefix='eeg2_attention', batch_size=128)

In [ ]:
filepath = 'results_data_FINAL.txt'

In [ ]:
f = open(filepath)
accList=list()
precisionList=list()
recallList=list()
f1List=list()

for line in f:
    if(str(line[0:13])=='Fold Accuracy'):
        listStart1=line.find('[')
        listEnd1=line.find(']')
        list1=ast.literal_eval(line[listStart1:listEnd1+1])
        accList.append(list1)
    if(str(line[0:14])=='Fold Precision'):
        listStart2=line.find('[')
        listEnd2=line.find(']')
        list2=ast.literal_eval(line[listStart2:listEnd2+1])
        precisionList.append(list2)
    if(str(line[0:11])=='Fold Recall'):
        listStart3=line.find('[')
        listEnd3=line.find(']')
        list3=ast.literal_eval(line[listStart3:listEnd3+1])
        recallList.append(list3)
    if(str(line[0:7])=='Fold F1'):
        listStart4=line.find('[')
        listEnd4=line.find(']')
        list4=ast.literal_eval(line[listStart4:listEnd4+1])
        f1List.append(list4)
f.close()

In [ ]:
allAccuracies=np.array(accList)
averageAccuracy=np.average(allAccuracies, axis=0)

In [ ]:
allPrecisions=np.array(precisionList)
averagePrecisions=np.average(allPrecisions, axis=0)

In [ ]:
allRecall=np.array(recallList)
averageRecall=np.average(allRecall, axis=0)

In [ ]:
allF1s=np.array(f1List)
averageF1s=np.average(allF1s, axis=0)

In [ ]:
with open(resultsFilename, "a") as text_file:
    print(f"Average Accuracies List: {str(averageAccuracy)}", file=text_file)
    print(f"Average Precisions List: {str(averagePrecisions)}", file=text_file)
    print(f"Average Recalls List: {str(averageRecall)}", file=text_file)
    print(f"Average F1 List: {str(averageF1s)}", file=text_file)